In [ ]:
import pickle
from sklearn.model_selection import train_test_split
import keras
import numpy as np
from keras import Sequential
from keras import callbacks
from keras.utils import to_categorical
from keras.layers import Dropout, BatchNormalization, Flatten, Dense
from google.colab import drive

In [ ]:
np.random.seed(42)

In [ ]:
! gdown --id 1lvFeDE7qdwYTR9NVqBnoYGpXcPPaZUf9    # upload data (embeddings) from Google Drive 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1lvFeDE7qdwYTR9NVqBnoYGpXcPPaZUf9
To: /content/img_embeddings.pkl
100% 3.54M/3.54M [00:00<00:00, 136MB/s]


In [ ]:
! gdown --id 1DJ6tWI6jJxFwNaQv_6NUGfSkGR_BALc8     # upload data (labels) from Google Drive

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DJ6tWI6jJxFwNaQv_6NUGfSkGR_BALc8
To: /content/img_labels.pkl
100% 27.8k/27.8k [00:00<00:00, 22.9MB/s]


In [ ]:
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def get_data(file_path: str) -> np.ndarray:
    with open(file_path, 'rb') as file:
        return pickle.load(file)

In [ ]:
embeddings = get_data('/content/img_embeddings.pkl')
labels = get_data('/content/img_labels.pkl')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
print(f'Training sample size: {len(X_train)}')
print(f'Test sample size: {len(X_test)}')

Training sample size: 2768
Test sample size: 692


In [ ]:
y_train = to_categorical(y_train, 233)
y_test = to_categorical(y_test, 233)

In [ ]:
model = Sequential([])
model.add(Dense(128, input_shape=(128, ), name='input_layer'))
model.add(Dense(150, activation='elu', name='hidden_layer_1'))
model.add(Dropout(0.4, name='dropout_1'))
model.add(Dense(150, activation='elu', name='hidden_layer_2'))
model.add(Dense(150, activation='elu', name='hidden_layer_3'))
model.add(BatchNormalization(name='batch_normalization'))
model.add(Dense(150, activation='elu', name='hidden_layer_4'))
model.add(Dropout(0.4, name='dropout_2'))
model.add(Dense(233, activation='softmax', name='output_layer'))

In [ ]:
callbacks = [
    callbacks.EarlyStopping(monitor='loss', min_delta=0.01, patience=13, verbose=1),  
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, min_delta=0.01, min_lr=1e-10, patience=5, verbose=1, mode='auto')
]

In [ ]:
model.compile(metrics=['accuracy'], loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_train, y_train, batch_size=72, epochs=70, validation_split=0.2,  callbacks=callbacks)

Epoch 1/70
31/31 [==============================] - 2s 19ms/step - loss: 5.5440 - accuracy: 0.0108 - val_loss: 5.3534 - val_accuracy: 0.0325 - lr: 0.0010
Epoch 2/70
31/31 [==============================] - 0s 12ms/step - loss: 4.5603 - accuracy: 0.0926 - val_loss: 5.1822 - val_accuracy: 0.1661 - lr: 0.0010
Epoch 3/70
31/31 [==============================] - 0s 10ms/step - loss: 3.7314 - accuracy: 0.2344 - val_loss: 4.9254 - val_accuracy: 0.3321 - lr: 0.0010
Epoch 4/70
31/31 [==============================] - 0s 11ms/step - loss: 3.0269 - accuracy: 0.3546 - val_loss: 4.5595 - val_accuracy: 0.4747 - lr: 0.0010
Epoch 5/70
31/31 [==============================] - 0s 10ms/step - loss: 2.4808 - accuracy: 0.4444 - val_loss: 4.1212 - val_accuracy: 0.5830 - lr: 0.0010
Epoch 6/70
31/31 [==============================] - 0s 10ms/step - loss: 2.0441 - accuracy: 0.5375 - val_loss: 3.6319 - val_accuracy: 0.6426 - lr: 0.0010
Epoch 7/70
31/31 [==============================] - 0s 10ms/step - loss: 1.7

In [ ]:
print(f'Metric on test: {model.evaluate(X_test, y_test)}')
print(f'Metric on train: {model.evaluate(X_train, y_train)}')

22/22 [==============================] - 0s 3ms/step - loss: 0.6174 - accuracy: 0.8801
Metric on test: [0.6174319386482239, 0.8800578117370605]
87/87 [==============================] - 0s 3ms/step - loss: 0.1107 - accuracy: 0.9740
Metric on train: [0.11070176213979721, 0.97398841381073]


In [235]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Dense)         (None, 128)               16512     
                                                                 
 hidden_layer_1 (Dense)      (None, 150)               19350     
                                                                 
 dropout_1 (Dropout)         (None, 150)               0         
                                                                 
 hidden_layer_2 (Dense)      (None, 150)               22650     
                                                                 
 hidden_layer_3 (Dense)      (None, 150)               22650     
                                                                 
 batch_normalization (BatchN  (None, 150)              600       
 ormalization)                                                   
                                                     

In [ ]:
model_file_path = '/content/drive/My Drive/Colab Notebooks/Data/dl_model'
model.save(model_file_path)